### Предобработка текста

In [ ]:
!pip install "numpy<1.26" --upgrade

In [ ]:
!pip install --force-reinstall pandas gensim

In [1]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
df = pd.read_csv("https://storage.yandexcloud.net/auth-def-2024/datasets/meta_table_with_texts.csv")
df = df[['author', 'text']]
df.head()

,author,text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...


Загружаем лемматизированные тексты из заранее подготовленного файла, сформированного после экспериментов.

In [3]:
df_lemm = pd.read_csv('https://storage.yandexcloud.net/auth-def-2024/datasets/lemm_texts.csv')
df_lemm.head()

,author,lemm_text
0,Пушкин Александр Сергеевич,литературный альбомъ сразить рыцарь послднимъ ...
1,Карамзин Николай Михайлович,достоинство древний новый перевод немецкий нек...
2,Гоголь Николай Васильевич,полный собрание сочинение письмо так переписка...
3,Мамин-Сибиряк Дмитрий Наркисович,собрана сочиненйтомъ восьмой издан марксъ петр...
4,Мамин-Сибиряк Дмитрий Наркисович,нимфа щегольский волжский пароход вулкан дать ...


In [4]:
df['lemm_text'] = df_lemm['lemm_text']
df.head()

,author,text,lemm_text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ сразить рыцарь послднимъ ...
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,достоинство древний новый перевод немецкий нек...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,полный собрание сочинение письмо так переписка...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,собрана сочиненйтомъ восьмой издан марксъ петр...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,нимфа щегольский волжский пароход вулкан дать ...


In [5]:
# Удаление невалидной записи с английским текстом
df = df.drop(index=1918)
df = df.reset_index(drop=True)
len(df)

2564

In [6]:
# Токенизация текста
def words_list(x):
  # Подсчёт слов в тексте
  words = re.findall(r'\b\w+\b', x)
  return [w.lower() for w in words]

In [7]:
df['words'] = df['lemm_text'].apply(words_list)
df.head(5)

,author,text,lemm_text,words
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ сразить рыцарь послднимъ ...,"[литературный, альбомъ, сразить, рыцарь, послд..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,достоинство древний новый перевод немецкий нек...,"[достоинство, древний, новый, перевод, немецки..."
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,полный собрание сочинение письмо так переписка...,"[полный, собрание, сочинение, письмо, так, пер..."
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,собрана сочиненйтомъ восьмой издан марксъ петр...,"[собрана, сочиненйтомъ, восьмой, издан, марксъ..."
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,нимфа щегольский волжский пароход вулкан дать ...,"[нимфа, щегольский, волжский, пароход, вулкан,..."


Загрузим эвристики, которые были рассчитаны и отобраны на этапе проведения экспериментов и замера метрик качества моделей.

In [8]:
# Загрузка фрейма с эвристиками
df_heuristics = pd.read_csv('https://storage.yandexcloud.net/auth-def-2024/datasets/heuristics.csv')
df_heuristics.head(5)

,avg_tonality,avg_subjectivity,total_words,avg_words_per_sentence
0,0.066585,0.229008,353,15.347826
1,-0.232829,0.358885,1008,15.750000
2,-0.010425,0.195767,196245,7.540634
3,-0.000427,0.007812,9247,8.353207
4,-0.000427,0.007812,4477,6.962675


### Формирование выборок

На этапе проведения экспериментов было принято решение использовать комбинированный подход в выборе признаков.

In [9]:
X_heuristics = df_heuristics.iloc[:2564].copy()
X_words = df['words'].copy()
y = df['author'].copy()

In [10]:
print("X_heuristics:", X_heuristics.shape)
print("X_words:", X_words.shape)
print("y:", y.shape)

X_heuristics: (2564, 4)
X_words: (2564,)
y: (2564,)


In [11]:
from sklearn.model_selection import train_test_split

# Обучающая и тренировочная выборки
X_heur_train, X_heur_test, X_words_train, X_words_test, y_train, y_test = train_test_split(X_heuristics, X_words, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# Логнормируем колонку total_words
X_heur_train['total_words'] = np.log1p(X_heur_train['total_words'])
X_heur_test['total_words']  = np.log1p(X_heur_test['total_words'])

# Отмасштабируем признаки
scaler = StandardScaler()
X_train_heur_scaled = scaler.fit_transform(X_heur_train)
X_test_heur_scaled  = scaler.transform(X_heur_test)

Обучение модели `Word2Vec` с использованием `SkipGram` для дальнейшего формирования эмбеддингов на тренировочных данных:

In [13]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=X_words_train, vector_size=300, window=5, min_count=1, workers=4, sg=1)

In [14]:
import numpy as np

# Функция для представления текста как среднего из векторов слов
def vectorize_text(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)  # Если нет слов из текста, возвращаем нулевой вектор
    return np.mean(vectors, axis=0)

In [15]:
X_train_vect = np.array([vectorize_text(text, w2v_model) for text in X_words_train])
X_test_vect = np.array([vectorize_text(text, w2v_model) for text in X_words_test])
X_train_vect = normalize(X_train_vect, norm='l2', axis=1)
X_test_vect  = normalize(X_test_vect,  norm='l2', axis=1)

In [16]:
# Формирование итоговых датасетов
X_train_combined = np.hstack([X_train_heur_scaled, X_train_vect])
X_test_combined = np.hstack([X_test_heur_scaled, X_test_vect])

In [17]:
# Словарь и embedding_matrix для RNN/CNN
word_to_index = {"<PAD>": 0, "<UNK>": 1}
for i, w in enumerate(w2v_model.wv.index_to_key, start=2):
    word_to_index[w] = i
vocab_size  = len(word_to_index)
embed_dim   = w2v_model.vector_size
emb_matrix  = np.zeros((vocab_size, embed_dim), dtype=np.float32)
emb_matrix[1] = np.random.normal(scale=0.1, size=(embed_dim,))
for w,i in word_to_index.items():
    if w in w2v_model.wv:
        emb_matrix[i] = w2v_model.wv[w]

# Последовательности фиксированной длины
max_len = 300
def to_seq(words):
    seq = [word_to_index.get(w,1) for w in words[:max_len]]
    seq += [0]*(max_len - len(seq))
    return seq
X_train_seq = np.array([to_seq(w) for w in X_words_train], dtype=np.int64)
X_test_seq  = np.array([to_seq(w) for w in X_words_test],  dtype=np.int64)
# эвристики переводим в float32
X_train_heur = X_train_heur_scaled.astype(np.float32)
X_test_heur  = X_test_heur_scaled.astype(np.float32)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
# Кодирование меток
le = LabelEncoder()
y_tr_enc = le.fit_transform(y_train)
y_te_enc = le.transform(y_test)
n_classes = len(le.classes_)

### Эксперименты с простыми DL моделями

#### Feed-forward Neural Network

Первая модель – FF-Net (Feed-forward Neural Network) - многослойная полносвязная нейронная сеть для табличных пользовательских признаков.

- Архитектура:
	1.	Входной слой = размер входного вектора (эвристики + средний Word2Vec)
	2.	Dense(256) → ReLU → Dropout(0.4)
	3.	Dense(128) → ReLU → Dropout(0.3)
	4.	Выходной Dense(n_classes) без активации (логиты для CrossEntropyLoss)

С помощью этой модели проверяем, как полносвязная сеть умеет работать с той же посчитанной матрицей признаков, что и XGBoost, но в виде нейросети; позволяет учиться нелинейным сочетаниям эвристик и эмбеддингов.

In [ ]:
# 1. Масштабирование комбинированных признаков
scaler_ff = StandardScaler()
X_tr_ff_np = scaler_ff.fit_transform(X_train_combined)
X_te_ff_np = scaler_ff.transform(X_test_combined)

X_tr_ff = torch.tensor(X_tr_ff_np, dtype=torch.float32).to(device)
y_tr_ff = torch.tensor(y_tr_enc,     dtype=torch.long).to(device)
X_te_ff = torch.tensor(X_te_ff_np,   dtype=torch.float32).to(device)

# 2. Определение модели Feed-forward Neural Network на PyTorch
class FFNet(nn.Module):
    def __init__(self, in_dim, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, n_out)
        )
    def forward(self, x):
        return self.net(x)

# 3. Функция обучения модели (Модель, оптимизатор, лосс и scheduler)
lrs = [1e-1, 1e-2, 1e-3, 5e-4, 1e-4, 5e-5]
results_ff = []
def train_ff(lr, weight_decay=1e-5, epochs=20):
    model = FFNet(X_tr_ff.shape[1], n_classes).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        opt.zero_grad()
        logits = model(X_tr_ff)
        loss   = crit(logits, y_tr_ff)
        loss.backward()
        opt.step()
    return model

# 4. Обучение модели на разных lr
for lr in lrs:
    m = train_ff(lr, weight_decay=1e-5, epochs=20)
    m.eval()
    with torch.no_grad():
        pred = m(X_te_ff).argmax(dim=1).cpu().numpy()
    results_ff.append({
        'lr': lr,
        'Precision': precision_score(y_te_enc, pred, average='weighted'),
        'Recall':    recall_score(y_te_enc,    pred, average='weighted'),
        'F1':        f1_score(y_te_enc,        pred, average='weighted'),
        'Accuracy':  accuracy_score(y_te_enc,  pred)
    })

In [ ]:
result_ff_df = pd.DataFrame(results_ff)
result_ff_df

,lr,Precision,Recall,F1,Accuracy
0,0.10000,0.023118,0.152047,0.040134,0.152047
1,0.01000,0.782310,0.779727,0.776312,0.779727
2,0.00100,0.637295,0.656920,0.639153,0.656920
3,0.00050,0.524865,0.539961,0.476933,0.539961
4,0.00010,0.249885,0.345029,0.255090,0.345029
5,0.00005,0.243479,0.278752,0.215900,0.278752


> Лучший learning-rate этой модели: 0.01

In [ ]:
print("Best FF  precision:", result_ff_df['Precision'].max())
print("Best FF  recall:   ", result_ff_df['Recall'].max())
print("Best FF  F1:       ", result_ff_df['F1'].max())
print("Best FF  accuracy:", result_ff_df['Accuracy'].max())

Best FF  precision: 0.7823100965650744
Best FF  recall:    0.7797270955165692
Best FF  F1:        0.7763124537990285
Best FF  accuracy: 0.7797270955165692


#### Bidirectional LSTM

Следующая модель - Bi-LSTM (Bidirectional LSTM) - рекуррентная нейросеть, учитывающая порядок слов и двунаправленный контекст.

- Архитектура:
	1.	Embedding-слой, инициализированный предобученными векторами Word2Vec (размерность 300), веса дообучаются
	2.	Однослойный двунаправленный LSTM с hidden_size=128 → Dropout(0.3)
	3.	Из выходных состояний берутся два вектора (последнее прямое и первое обратное состояние), конкатенируются → Dense(n_classes)
	4.	Вход в финальный Dense объединён с эвристическими признаками

Модель «читает» текст как последовательность, улавливает характерные для автора n-граммы и порядок слов, а двунаправленность усиливает представление начала и конца текста.

In [ ]:
# 1. Формируем датасет
ds_tr = TensorDataset(
    torch.tensor(X_train_seq),
    torch.tensor(X_train_heur),
    torch.tensor(y_tr_enc)
)
loader = DataLoader(ds_tr, batch_size=16, shuffle=True, pin_memory=True)

# 2. Определяем модель
class BiLSTMNet(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, n_out, emb_w):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.emb.weight.data.copy_(torch.tensor(emb_w))
        self.emb.weight.requires_grad = True
        self.lstm = nn.LSTM(emb_dim, hid_dim, batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(0.3)
        self.fc   = nn.Linear(hid_dim*2 + X_train_heur.shape[1], n_out)

    def forward(self, seq, heur):
        x, _ = self.lstm(self.emb(seq))
        h = torch.cat([x[:, -1, :self.lstm.hidden_size],
                       x[:, 0, self.lstm.hidden_size:]], 1)
        h = self.drop(h)
        out = self.fc(torch.cat([h, heur],1))
        return out

# 3. Задаем параметры обучения
lrs = [1e-1, 1e-2, 1e-3, 5e-4, 1e-4, 5e-5]
results_lstm = []

def train_lstm(lr, weight_decay=1e-5, epochs=10):
    model = BiLSTMNet(vocab_size, embed_dim, 128, n_classes, emb_matrix).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.CrossEntropyLoss()
    model.train()
    for _ in range(epochs):
        for xb_seq, xb_heu, yb in loader:
            opt.zero_grad()
            xb_seq = xb_seq.to(device)
            xb_heu = xb_heu.to(device)
            yb     = yb.to(device)
            out = model(xb_seq, xb_heu)
            loss = crit(out, yb)
            loss.backward()
            opt.step()
    return model

# 4. Обучаем и формируем оценки предсказаний
for lr in lrs:
    m = train_lstm(lr, weight_decay=1e-5, epochs=15)
    m.eval()
    with torch.no_grad():
        X_seq_te = torch.tensor(X_test_seq, dtype=torch.long).to(device)
        X_heur_te= torch.tensor(X_test_heur, dtype=torch.float32).to(device)
        out = m(X_seq_te, X_heur_te)
        pred = out.argmax(dim=1).cpu().numpy()

    results_lstm.append({
        'lr': lr,
        'Precision': precision_score(y_te_enc, pred, average='weighted'),
        'Recall':    recall_score(y_te_enc,    pred, average='weighted'),
        'F1':        f1_score(y_te_enc,        pred, average='weighted'),
        'Accuracy':  accuracy_score(y_te_enc,  pred)
    })

In [26]:
df_lstm = pd.DataFrame(results_lstm)
df_lstm

,lr,Precision,Recall,F1,Accuracy
0,0.10000,0.002189,0.046784,0.004182,0.046784
1,0.01000,0.762976,0.732943,0.738503,0.732943
2,0.00100,0.760938,0.744639,0.747074,0.744639
3,0.00050,0.775842,0.756335,0.762057,0.756335
4,0.00010,0.662725,0.674464,0.657460,0.674464
5,0.00005,0.585570,0.614035,0.570117,0.614035


> Лучший learning-rate этой модели: 0.001

In [27]:
print("Best Bi-LSTM Precision:", df_lstm['Precision'].max())
print("Best Bi-LSTM Recall:   ", df_lstm['Recall'].max())
print("Best Bi-LSTM F1:       ", df_lstm['F1'].max())
print("Best Bi-LSTM Accuracy:", df_lstm['Accuracy'].max())

Best Bi-LSTM Precision: 0.775842404883158
Best Bi-LSTM Recall:    0.7563352826510721
Best Bi-LSTM F1:        0.7620569009783128
Best Bi-LSTM Accuracy: 0.7563352826510721


#### TensorFlow MultiLayer Perceptron

Последняя модель которую тестируем здесь - TF-MLP (Multilayer Perceptron на TensorFlow/Keras) - прямое сравнение с PyTorch-MLP на тех же признаках.

Модель - простой MLP в Keras для быстрой проверки работы альтернативного фреймворка.

- Архитектура:
	1.	Входной Dense-слой на 128 нейронов с активацией ReLU
	2.	Dropout(0.3)
	3.	Выходной Dense(n_classes) с softmax

In [ ]:
# 1. Масштабирование комбинированных признаков
scaler_cf = StandardScaler()
X_tr_tf = scaler_cf.fit_transform(X_train_combined)
X_te_tf = scaler_cf.transform(X_test_combined)

# 2. Задаем диапазон learning rates и девайс для tensorflow
lrs = [1e-1, 1e-2, 1e-3, 5e-4, 1e-4, 5e-5]
results_tf = []
device_tf = "/GPU:0" if tf.config.list_physical_devices("GPU") else "/CPU:0"

# 3. Обучение
for lr in lrs:
    with tf.device(device_tf):
        model = keras.Sequential([
            layers.Input(shape=(X_tr_tf.shape[1],)),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(n_classes, activation='softmax'),
        ])
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=lr),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        model.fit(X_tr_tf, y_tr_enc, epochs=10, batch_size=32, verbose=0)
        y_pred = model.predict(X_te_tf).argmax(axis=1)

    results_tf.append({
        'lr': lr,
        'Precision': precision_score(y_te_enc, y_pred, average='weighted'),
        'Recall':    recall_score(y_te_enc,    y_pred, average='weighted'),
        'F1':        f1_score(y_te_enc,        y_pred, average='weighted'),
        'Accuracy':  accuracy_score(y_te_enc,  y_pred)
    })

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
df_tf = pd.DataFrame(results_tf)
df_tf

,lr,Precision,Recall,F1,Accuracy
0,0.10000,0.816144,0.803119,0.804594,0.803119
1,0.01000,0.828043,0.826511,0.821359,0.826511
2,0.00100,0.868236,0.867446,0.865639,0.867446
3,0.00050,0.816001,0.826511,0.820202,0.826511
4,0.00010,0.659670,0.668616,0.651787,0.668616
5,0.00005,0.547486,0.576998,0.545915,0.576998


> Лучший learning-rate этой модели: 0.001

In [ ]:
print("Best TF-MLP Precision:", df_tf['Precision'].max())
print("Best TF-MLP Recall:   ", df_tf['Recall'].max())
print("Best TF-MLP F1:       ", df_tf['F1'].max())
print("Best TF-MLP Accuracy:", df_tf['Accuracy'].max())

Best TF-MLP Precision: 0.8682359682884365
Best TF-MLP Recall:    0.8674463937621832
Best TF-MLP F1:        0.8656385399242229
Best TF-MLP Accuracy: 0.8674463937621832


### Эксперименты с рекуррентными нейронными сетями

Ранее был рассмотрен подход $\text{Bidirectional LSTM}$, ниже рассмотрим прочие варианты применения рекуррентных сетей в нашей задаче:

#### Stacked RNN

Является модифицированной версией обычного $\text{LSTM}$, так как вместо одного слоя используется несколько подряд. Так как мы используем достаточно большие тексты, использование такого варианта может быть полезным - чем больше проходок, тем лучше идентифицируется стилистика.<br>
Рассмотрим вариант с увеличением слоев `num_layers=3` (так как есть большие тексты):

In [ ]:
# 1. Формируем датасет
ds_tr = TensorDataset(
    torch.tensor(X_train_seq),
    torch.tensor(X_train_heur),
    torch.tensor(y_tr_enc)
)
loader = DataLoader(ds_tr, batch_size=16, shuffle=True, pin_memory=True)

# 2. Определяем модель
class StackedBiLSTMNet(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, n_out, emb_w, num_layers=3):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.emb.weight.data.copy_(torch.tensor(emb_w))
        self.emb.weight.requires_grad = True

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=num_layers,  # Задаём количество слоёв
            batch_first=True,
            bidirectional=True,
            dropout=0.3 if num_layers > 1 else 0.0
        )

        self.drop = nn.Dropout(0.3)
        self.fc   = nn.Linear(hid_dim*2 + X_train_heur.shape[1], n_out)

    def forward(self, seq, heur):
        x = self.emb(seq)
        x, _ = self.lstm(x)
        h = torch.cat([x[:, -1, :self.lstm.hidden_size],
                       x[:, 0, self.lstm.hidden_size:]], dim=1)
        h = self.drop(h)
        out = self.fc(torch.cat([h, heur], dim=1))
        return out

# 3. Задаем параметры обучения
lrs = [1e-1, 1e-2, 1e-3, 5e-4, 1e-4, 5e-5]
results_lstm_stacked = []

def train_lstm_stacked(lr, weight_decay=1e-5, epochs=10):
    model = StackedBiLSTMNet(vocab_size, embed_dim, 128, n_classes, emb_matrix).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.CrossEntropyLoss()
    model.train()
    for _ in range(epochs):
        for xb_seq, xb_heu, yb in loader:
            opt.zero_grad()
            xb_seq = xb_seq.to(device)
            xb_heu = xb_heu.to(device)
            yb     = yb.to(device)
            out = model(xb_seq, xb_heu)
            loss = crit(out, yb)
            loss.backward()
            opt.step()
    return model

# 4. Обучаем и формируем оценки предсказаний
for lr in lrs:
    m = train_lstm_stacked(lr, weight_decay=1e-5, epochs=10)
    m.eval()
    with torch.no_grad():
        X_seq_te = torch.tensor(X_test_seq, dtype=torch.long).to(device)
        X_heur_te= torch.tensor(X_test_heur, dtype=torch.float32).to(device)
        out = m(X_seq_te, X_heur_te)
        pred = out.argmax(dim=1).cpu().numpy()

    results_lstm_stacked.append({
        'lr': lr,
        'Precision': precision_score(y_te_enc, pred, average='weighted'),
        'Recall':    recall_score(y_te_enc,    pred, average='weighted'),
        'F1':        f1_score(y_te_enc,        pred, average='weighted'),
        'Accuracy':  accuracy_score(y_te_enc,  pred)
    })

In [21]:
df_lstm_stacked = pd.DataFrame(results_lstm_stacked)
df_lstm_stacked

,lr,Precision,Recall,F1,Accuracy
0,0.10000,0.002189,0.046784,0.004182,0.046784
1,0.01000,0.702559,0.645224,0.660632,0.645224
2,0.00100,0.764541,0.742690,0.749095,0.742690
3,0.00050,0.778969,0.727096,0.746767,0.727096
4,0.00010,0.733563,0.705653,0.712633,0.705653
5,0.00005,0.636902,0.656920,0.637081,0.656920


In [22]:
print("Best Stacked Bi-LSTM Precision:", df_lstm_stacked['Precision'].max())
print("Best Stacked Bi-LSTM Recall:   ", df_lstm_stacked['Recall'].max())
print("Best Stacked Bi-LSTM F1:       ", df_lstm_stacked['F1'].max())
print("Best Stacked Bi-LSTM Accuracy: ", df_lstm_stacked['Accuracy'].max())

Best Stacked Bi-LSTM Precision: 0.7789694056996576
Best Stacked Bi-LSTM Recall:    0.7426900584795322
Best Stacked Bi-LSTM F1:        0.749095094045158
Best Stacked Bi-LSTM Accuracy:  0.7426900584795322


Несмотря на предположение об улучшении способности распознавать стили текстов, качество упало в сравнении с исходным одним слоем $\text{Bi-LSTM}$.

#### RNN + Attention

В следующей вариации добавим поверх выходов $\text{Bi-LSTM}$ *механизм внимания* ($\text{Attention}$), который научится выделять важные слова в тексте для финального решения. Реализация заключается в применении attention-слоя после LSTM и дальнейшей агрегацией скрытых состояния с весами важности:

In [ ]:
# 1. Формируем датасет
ds_tr = TensorDataset(
    torch.tensor(X_train_seq),
    torch.tensor(X_train_heur),
    torch.tensor(y_tr_enc)
)
loader = DataLoader(ds_tr, batch_size=16, shuffle=True, pin_memory=True)

# 2. Определяем модель
class BiLSTMWithAttentionNet(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, n_out, emb_w):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.emb.weight.data.copy_(torch.tensor(emb_w))
        self.emb.weight.requires_grad = True

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hid_dim,
            batch_first=True,
            bidirectional=True
        )

        self.attention = nn.Linear(hid_dim * 2, 1)  # Attention по каждому скрытому слою
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(hid_dim * 2 + X_train_heur.shape[1], n_out)

    def forward(self, seq, heur):
        x = self.emb(seq)
        rnn_out, _ = self.lstm(x)

        # Attention механизм
        attn_scores = self.attention(rnn_out).squeeze(2)
        attn_weights = torch.softmax(attn_scores, dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), rnn_out).squeeze(1)

        h = self.drop(attn_applied)
        out = self.fc(torch.cat([h, heur], dim=1))
        return out

# 3. Задаем параметры обучения
lrs = [1e-1, 1e-2, 1e-3, 5e-4, 1e-4, 5e-5]
results_lstm_attention = []

def train_lstm_attention(lr, weight_decay=1e-5, epochs=10):
    model = BiLSTMWithAttentionNet(vocab_size, embed_dim, 128, n_classes, emb_matrix).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.CrossEntropyLoss()
    model.train()
    for _ in range(epochs):
        for xb_seq, xb_heu, yb in loader:
            opt.zero_grad()
            xb_seq = xb_seq.to(device)
            xb_heu = xb_heu.to(device)
            yb     = yb.to(device)
            out = model(xb_seq, xb_heu)
            loss = crit(out, yb)
            loss.backward()
            opt.step()
    return model

# 4. Обучаем и формируем оценки предсказаний
for lr in lrs:
    m = train_lstm_attention(lr, weight_decay=1e-5, epochs=10)
    m.eval()
    with torch.no_grad():
        X_seq_te = torch.tensor(X_test_seq, dtype=torch.long).to(device)
        X_heur_te= torch.tensor(X_test_heur, dtype=torch.float32).to(device)
        out = m(X_seq_te, X_heur_te)
        pred = out.argmax(dim=1).cpu().numpy()

    results_lstm_attention.append({
        'lr': lr,
        'Precision': precision_score(y_te_enc, pred, average='weighted'),
        'Recall':    recall_score(y_te_enc,    pred, average='weighted'),
        'F1':        f1_score(y_te_enc,        pred, average='weighted'),
        'Accuracy':  accuracy_score(y_te_enc,  pred)
    })

In [45]:
df_lstm_attention = pd.DataFrame(results_lstm_attention)
df_lstm_attention

,lr,Precision,Recall,F1,Accuracy
0,0.10000,0.555358,0.516569,0.497102,0.516569
1,0.01000,0.794858,0.785575,0.785592,0.785575
2,0.00100,0.787545,0.787524,0.785529,0.787524
3,0.00050,0.785017,0.768031,0.770510,0.768031
4,0.00010,0.544710,0.575049,0.523568,0.575049
5,0.00005,0.430913,0.469786,0.400070,0.469786


In [47]:
print("Best Bi-LSTM with Attention Precision:", df_lstm_attention['Precision'].max())
print("Best Bi-LSTM with Attention Recall:   ", df_lstm_attention['Recall'].max())
print("Best Bi-LSTM with Attention F1:       ", df_lstm_attention['F1'].max())
print("Best Bi-LSTM with Attention Accuracy: ", df_lstm_attention['Accuracy'].max())

Best Bi-LSTM with Attention Precision: 0.7948583097463461
Best Bi-LSTM with Attention Recall:    0.7875243664717348
Best Bi-LSTM with Attention F1:        0.7855916736370059
Best Bi-LSTM with Attention Accuracy:  0.7875243664717348


С добавлением механизма важности получилось побить качество на исходной рекуррентной нейронной сети без attention.

#### RCNN

Заключительным вариантом использования рекуррентных нейронных сетей будет их комбинация со сверточными. Идея заключается в том, чтобы после LSTM (который строит контекст) прогнать скрытые состояния через свёрточный слой CNN, который выделит локальные устойчивые паттерны из предыдущих выходов.<br>
Рассмотрим `Conv1d` - свёртка по одному измерению, которая будет обобщать информацию, накопленную `LSTM` на уровне токенов. В качестве размера окна возьмем стандартное значение `kernel_size=3`:

In [ ]:
# 1. Формируем датасет
ds_tr = TensorDataset(
    torch.tensor(X_train_seq),
    torch.tensor(X_train_heur),
    torch.tensor(y_tr_enc)
)
loader = DataLoader(ds_tr, batch_size=16, shuffle=True, pin_memory=True)

# 2. Определяем модель
class RCNNNet(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, n_out, emb_w, cnn_kernel_size=3):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.emb.weight.data.copy_(torch.tensor(emb_w))
        self.emb.weight.requires_grad = True

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hid_dim,
            batch_first=True,
            bidirectional=True
        )

        self.conv1d = nn.Conv1d(
            in_channels=hid_dim * 2,
            out_channels=hid_dim * 2,
            kernel_size=cnn_kernel_size,
            padding=cnn_kernel_size // 2
        )

        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(hid_dim * 2 + X_train_heur.shape[1], n_out)

    def forward(self, seq, heur):
        x = self.emb(seq)
        rnn_out, _ = self.lstm(x)
        rnn_out = rnn_out.permute(0, 2, 1)
        conv_out = self.conv1d(rnn_out)
        conv_out = torch.relu(conv_out)
        conv_out = conv_out.permute(0, 2, 1)
        pooled = conv_out.mean(dim=1)

        h = self.drop(pooled)
        out = self.fc(torch.cat([h, heur], dim=1))
        return out

# 3. Задаем параметры обучения
lrs = [1e-1, 1e-2, 1e-3, 5e-4, 1e-4, 5e-5]
results_rcnn = []

def train_rcnn(lr, weight_decay=1e-5, epochs=10):
    model = RCNNNet(vocab_size, embed_dim, 128, n_classes, emb_matrix).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.CrossEntropyLoss()
    model.train()
    for _ in range(epochs):
        for xb_seq, xb_heu, yb in loader:
            opt.zero_grad()
            xb_seq = xb_seq.to(device)
            xb_heu = xb_heu.to(device)
            yb     = yb.to(device)
            out = model(xb_seq, xb_heu)
            loss = crit(out, yb)
            loss.backward()
            opt.step()
    return model

# 4. Обучаем и формируем оценки предсказаний
for lr in lrs:
    m = train_rcnn(lr, weight_decay=1e-5, epochs=10)
    m.eval()
    with torch.no_grad():
        X_seq_te = torch.tensor(X_test_seq, dtype=torch.long).to(device)
        X_heur_te= torch.tensor(X_test_heur, dtype=torch.float32).to(device)
        out = m(X_seq_te, X_heur_te)
        pred = out.argmax(dim=1).cpu().numpy()

    results_rcnn.append({
        'lr': lr,
        'Precision': precision_score(y_te_enc, pred, average='weighted'),
        'Recall':    recall_score(y_te_enc,    pred, average='weighted'),
        'F1':        f1_score(y_te_enc,        pred, average='weighted'),
        'Accuracy':  accuracy_score(y_te_enc,  pred)
    })

In [21]:
df_rcnn = pd.DataFrame(results_rcnn)
df_rcnn

,lr,Precision,Recall,F1,Accuracy
0,0.10000,0.541243,0.565302,0.511678,0.565302
1,0.01000,0.835295,0.812865,0.815129,0.812865
2,0.00100,0.743572,0.721248,0.718936,0.721248
3,0.00050,0.681162,0.623782,0.614303,0.623782
4,0.00010,0.511064,0.520468,0.507130,0.520468
5,0.00005,0.365365,0.456140,0.392211,0.456140


In [22]:
print("Best RCNN Precision:", df_rcnn['Precision'].max())
print("Best RCNN Recall:   ", df_rcnn['Recall'].max())
print("Best RCNN F1:       ", df_rcnn['F1'].max())
print("Best RCNN Accuracy: ", df_rcnn['Accuracy'].max())

Best RCNN Precision: 0.8352945246322128
Best RCNN Recall:    0.8128654970760234
Best RCNN F1:        0.8151293518107349
Best RCNN Accuracy:  0.8128654970760234


Данный вариант применения рекуррентных нейронных сетей показал еще более высокий результат, чем способ с механизмом важности, значительно улучшив метрику в сравнении с базовым $\text{Bi-LSTM}$.

**ВЫВОД:** при подходе с рекуррентными нейронными сетями лучшее качество на метриках показал вариант комбинирования $\text{RNN + CNN}$, выдав $F1=0.815$.

### Что было до DL моделей

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier

# Кодирование таргета необходимо для корректной работы модели XGBoost
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

# Пайплайн
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(eval_metric='logloss', learning_rate=0.1, max_depth=3, n_estimators=200, subsample=0.8))
])

pipeline.fit(X_train_combined, y_train_enc)
y_pred_boost = pipeline.predict(X_test_combined)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

# Оценка качества модели бустинга
# Accuracy
accuracy = accuracy_score(y_test_enc, y_pred_boost)
print("Accuracy:", accuracy)

# Precision, Recall, F1 (взвешенные для многоклассовой задачи)
precision = precision_score(y_test_enc, y_pred_boost, average='weighted')
recall = recall_score(y_test_enc, y_pred_boost, average='weighted')
f1 = f1_score(y_test_enc, y_pred_boost, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test_enc, y_pred_boost)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.898635477582846
Precision: 0.8989564306728367
Recall: 0.898635477582846
F1 Score: 0.8975763500620028
Confusion Matrix:
 [[19  0  0  0  0  0  3  0  0  0  1  0  1]
 [ 0 44  0  0  0  0  0  0  1  0  1  0  1]
 [ 0  0 36  0  0  0  0  0  0  0  1  1  0]
 [ 0  0  1 34  0  0  1  0  0  1  1  2  0]
 [ 0  1  0  0 10  0  1  0  0  0  0  1  1]
 [ 0  1  1  0  0 54  0  0  0  1  0  0  0]
 [ 0  1  0  2  1  1 71  0  0  1  0  1  0]
 [ 0  0  0  0  0  0  1  6  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0 50  0  0  0  0]
 [ 2  0  1  0  0  2  0  0  0 21  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0 34  1  0]
 [ 0  0  0  1  0  0  1  0  2  0  0 55  1]
 [ 0  1  0  1  0  0  3  0  1  0  2  0 27]]


In [ ]:
print(classification_report(y_test_enc, y_pred_boost, target_names=list(y_train.unique())))

                                   precision    recall  f1-score   support

             Чехов Антон Павлович       0.90      0.79      0.84        24
        Куприн Александр Иванович       0.92      0.94      0.93        47
            Бунин Иван Алексеевич       0.92      0.95      0.94        38
      Карамзин Николай Михайлович       0.87      0.85      0.86        40
     Достоевский Федор Михайлович       0.91      0.71      0.80        14
         Лермонтов Михаил Юрьевич       0.95      0.95      0.95        57
       Пушкин Александр Сергеевич       0.87      0.91      0.89        78
 Мамин-Сибиряк Дмитрий Наркисович       1.00      0.86      0.92         7
        Гоголь Николай Васильевич       0.93      0.98      0.95        51
          Тургенев Иван Сергеевич       0.88      0.81      0.84        26
Салтыков-Щедрин Михаил Евграфович       0.85      0.94      0.89        36
     Блок Александр Александрович       0.90      0.92      0.91        60
      Есенин Сергей Алек

По итогу на комбинированном наборе фичей (эвристики + текстовые), а также при использовании word2vec модели со SkipGram, бустинг модель XGBoost показала гораздо лучшие значения метрик точности предсказания в районе $\approx 90\%$

(до этого значения метрик доходили до $\approx 81\%$ у модели логистической регрессии, и по $\approx 87-88\%$ для word2vec со skipgram и xgboost по отдельности).

### Результаты экспериментов

Было опробовано включить в пайплайн модели глубинного обучения (рассматривались три базовые модели двух разных фреймворков - pytorch и tensorflow, а также разные подходы с рекуррентными сетями).

Итоги экспериментов следующие:
 - Лучшей моделью по-прежнему остаётся `XGBoost` на комбинированных признаках (`≈ 90%` на F1).
 - Из базовых нейросетей к этой же метрике близка `TF-MLP` (`≈ 86.6%` на F1), тогда как `Bi-LSTM` и `FF-Net` показали результаты F1 равным `~76–78%`.
 - Среди подходов с рекуррентными сетями лучшим оказалась комбинация `RCNN`, показавшая метрику F1 равной `81.5%`.
 - Для дальнейшего подъёма качества можно рассмотреть более современные и сложные модели (например `DeepPavlov`), а также дообучение на современных трансформерах (`BERT/RuBERT`).